In [2]:
from riotwatcher import LolWatcher, ApiError
import riotwatcher
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import os
import shutil

In [8]:
api_key = 'RGAPI-df101462-9dcb-46e1-81a4-8da803b27924'
watcher = LolWatcher(api_key)
my_region = 'euw1'

In [12]:
summoners = watcher.match.by_id(my_region, watcher.match.matchlist_by_puuid(my_region, watcher.summoner.by_name(my_region,'alvarof98')['puuid'])[0])['metadata']['participants']
summoners = [watcher.summoner.by_puuid(my_region, i)['name'] for i in summoners]
summoners

['Ebers',
 'alvarof98',
 'xn88x',
 'Rachell94',
 'FRvsALL',
 'WhereEaglesDare',
 'TFS Senfei',
 'lordletters',
 'VBLODUS0secse',
 'Gamers2 Caps']

In [10]:
watcher.summoner.by_name(my_region, 'alvarof98')

{'id': 'P_4EjJGQdqY4-RwuDXli2f0YeI2fRiVLfSrbFWGfdHHd5t0',
 'accountId': 'MGX6g7z2rTZ8kvZ1OCD_clpRC6GFCYMnxaOJfpTUxA50DKQ',
 'puuid': '-6QqogKKl14ju1MZplivTIaNe9QgEkoR0kaEAfMHlAjPDKMNfUIpv5ihibq0Fx4P5MdYQIaqTLcvQg',
 'name': 'alvarof98',
 'profileIconId': 1115,
 'revisionDate': 1658350556000,
 'summonerLevel': 214}

In [6]:
textos = []
summoners = ['thedatawarlock']
for i in summoners:
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
    reponse = requests.get(f'https://www.leagueofgraphs.com/summoner/euw/{i}', headers = headers)
    text = ''
    soup = BeautifulSoup(reponse.text, 'html.parser')
    for j in soup.find(attrs = {'class':'box tags-box'}).find_all('div'):
        if 'brown' in j['class']:
            continue
        elif 'lover' in j.text:
            text+='lover '
        else:
            text += j.text.replace('\n','').strip(' ') + ' '
    textos.append(text)
    sleep(0.5)
textos

['lover lover Bounces back Snowballs Too confident Loses when behind Bad Starts ']

In [4]:
# Configuramos el modelo de sentimientos de hugging face

if 'cardiffnlp' in os.listdir():
    shutil.rmtree('cardiffnlp')
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast = False)
config = AutoConfig.from_pretrained(MODEL)

In [7]:
cont = 0
for i in textos:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.save_pretrained(MODEL)
    encoded_input = tokenizer(i, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    print(f'{summoners[cont]} Indice de toxicidad:')
    for j in range(scores.shape[0]):
        l = config.id2label[ranking[j]]
        s = scores[ranking[j]]
        print(f"{j+1}) {l} {np.round(float(s), 4)}")
    print('====================================')
    cont +=1

thedatawarlock Indice de toxicidad:
1) Negative 0.6891
2) Neutral 0.26
3) Positive 0.0509
